In [1]:
# useing this variable for saving models and summaries
exp_name = "HRNN_NewWavelet_Wmultitask_16_opp_20K"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../../src')

In [3]:
from models.HRNN_model_new_Wavelet import Model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [7]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

wavelet_levels = 7    #number of wavelet layer to have in the network
convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 2, 2]
weights_shape = [[5, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
wavelet_kernel_size = 3    
activation = tf.nn.relu

lstm_n_hidden = 64
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)

pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)

variables_weights = np.ones((n_classes,))

In [9]:
pos_weight.shape

(19,)

In [10]:
pos_weight

array([  0.36565482,  48.46756152,  47.65126513,  50.0669746 ,
        49.94930876,  40.02411874,  41.52307692,  58.84303112,
        63.4664723 ,  75.37996546,  86.74603175,  80.14495413,
        87.27145709,  69.42038217,  68.86413902,  43.71587462,
        13.05721551,  58.2021419 ,  10.05323669])

In [11]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [12]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/' + exp_name 

In [13]:
model = Model(n_variables = n_variables,
                            learning_rate = learning_rate,
                            n_classes = n_classes)

In [14]:
model.build_model(pos_weight = pos_weight, 
                 learning_rate = learning_rate,
                 wavelet_levels = wavelet_levels,    
                 convolution_levels = convolution_levels,    
                 pooling_stride = pooling_stride,     
                 weights_shape = weights_shape,
                 bias_shape = bias_shape,
                 wavelet_kernel_size = wavelet_kernel_size,    
                 activation = activation,
                 lstm_n_hidden = lstm_n_hidden,
                 trainable_wavelet = True,
                 trainable_conv = True,
                 trainable_last = True,
                 variables_weights = variables_weights
)


In [15]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [16]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Training: 0
ACTIVITY RECOGNITION
AUC = 0.062 vs. 0.062
F1 = 0.104 vs. 0.094
Accuracy = 0.009 vs. 0.056
CHANGE DETECTION
AUC = 0.094 vs. 0.102
F1 = 0.189 vs. 0.188
Precision = 0.106 vs. 0.106
Recall = 0.904 vs. 0.870
Validation: 0
ACTIVITY RECOGNITION
AUC = 0.062 vs. 0.060
F1 = 0.088 vs. 0.079
Accuracy = 0.017 vs. 0.056
CHANGE DETECTION
AUC = 0.048 vs. 0.049
F1 = 0.103 vs. 0.097
Precision = 0.054 vs. 0.051
Recall = 0.966 vs. 0.992
Minibatch Loss= 1.400876
Training: 100
ACTIVITY RECOGNITION
AUC = 0.364 vs. 0.061
F1 = 0.426 vs. 0.089
Accuracy = 0.440 vs. 0.056
CHANGE DETECTION
AUC = 0.220 vs. 0.094
F1 = 0.331 vs. 0.157
Precision = 0.257 vs. 0.087
Recall = 0.466 vs. 0.845
Validation: 100
ACTIVITY RECOGNITION
AUC = 0.181 vs. 0.060
F1 = 0.252 vs. 0.080
Accuracy = 0.426 vs. 0.056
CHANGE DETECTION
AUC = 0.175 vs. 0.048
F1 = 0.265 vs. 0.097
Precision = 0.184 vs. 0.051
Recall = 0.475 vs. 0.992
Minibatch Loss= 0.539392
Training: 200
ACTIVITY RECOGNITION
AUC = 0.554 vs. 0.060
F1 = 0.566 vs. 0.086


In [17]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model/HRNN_NewWavelet_Wmultitask_16_opp_20K/HRNN_NewWavelet_Wmultitask_16_opp_20K-20000'

In [17]:
#load the model
saver = tf.train.Saver()
saver.restore(sess, checkpoint_path+'-10000')

INFO:tensorflow:Restoring parameters from ../../model/HRNN_NewWavelet_Wmultitask_16_opp/HRNN_NewWavelet_Wmultitask_16_opp-10000


In [18]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

ACTIVITY RECOGNITION
AUC = 0.616 vs. 0.058
F1 = 0.623 vs. 0.079
Accuracy = 0.894 vs. 0.056
CHANGE DETECTION
AUC = 0.308 vs. 0.052
F1 = 0.373 vs. 0.097
Precision = 0.329 vs. 0.051
Recall = 0.431 vs. 0.958


In [19]:
out = nn_eval.deploy (model, sess, test_data, test_gt)

In [20]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
#probability_change = np.squeeze(probability_of_pos, axis = 2)

In [21]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':out[:,:,-1]})

In [ ]:
1

In [ ]:
2